<a href="https://colab.research.google.com/github/liujingiris52-dot/Assignment/blob/main/CodeSnippets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## MyDataset() for AlexNet and Vision Transformer ##

In [ ]:
### Custom my own dataset ###
class MyDataset(Dataset):
    def __init__(self, file_list):
        samples_list = []
        samples_label = []
        for sample_file in file_list: #
            samples_list.append(sample_file)
            if "dog" in sample_file: #
                samples_label.append(1)
            else:
                samples_label.append(0)
        self.data = np.array(samples_list)
        self.labels = np.array(samples_label)
        self.tensor = torchvision.transforms.Compose([torchvision.transforms.ToTensor()]) #
    def __getitem__(self, index): #
        img = self.data[index]
        label = self.labels[index]
        img = Image.open(img).convert('RGB') #
        data = self.tensor(img) #
        return data, label
    def __len__(self): #necessary function
        return self.data.shape[0]

## Train Loop train_model() for AlexNet and Vision Transformer ##

In [ ]:
### Design the Train Loop ###
def train_model(model, dataloaders, criterion, optimizer, lr_scheduler, num_epochs, device): #
    model.to(device)
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_epoch = 0
    train_loss_history = []
    train_auPR_history = []
    val_loss_history = []
    val_auPR_history = []

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        for phase in ['train', 'val']: # Switch training and validation phase #
            all_probs = []
            all_predics = []
            all_labels = []
            if phase == 'train': #
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]: # Iterate over data
                all_labels +=list(labels)
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad() #
                with torch.set_grad_enabled(phase == 'train'): #
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    probs = nn.functional.softmax(outputs,dim=1) #
                    _,preds = torch.max(probs, 1) #
                    if phase == 'train':
                        loss.backward() #
                        optimizer.step()
                # output performance of each epoch in validation dataset #
                all_probs.extend(probs.detach().cpu().numpy()[:,1])
                all_predics.extend(preds.cpu().numpy())
                # Statistics #
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            print('{} loss: {:.4f} Acc: {:.4f} '.format(phase, epoch_loss, epoch_acc))
            # performance #
            auROC = metrics.roc_auc_score(all_labels,all_probs)
            precision, recall, _ = metrics.precision_recall_curve(all_labels, all_probs)
            auPR = metrics.auc(recall, precision)
            precision = metrics.precision_score(all_labels,all_predics)
            recall = metrics.recall_score(all_labels,all_predics)
            f1 = metrics.f1_score(all_labels,all_predics)
            if phase == 'val':
                lr_scheduler.step(epoch_loss) #
                val_loss_history.append(epoch_loss)
                val_auPR_history.append(auPR)
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_epoch = epoch
                    best_model_wts = copy.deepcopy(model.state_dict()) #
            else:
                train_loss_history.append(epoch_loss)
                train_auPR_history.append(auPR)
            f = open("./results.txt","a")
            f.write('Phase: {} epoch: {:2d}/{} loss: {:.4f} Acc: {:.4f} auROC: {:.4f} auPR: {:.4f} precision: {:.4f} recall: {:.4f} f1: {:.4f} \n'.format(phase,epoch,num_epochs-1,epoch_loss,epoch_acc,auROC,auPR,precision,recall,f1))
            f.close()
        torch.save(model.state_dict(), "./checkpoints/epoch{}.pt".format(epoch))
        print(datetime.datetime.now())

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:.4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    torch.save(model.state_dict(), "./best_model/model.best_val_acc.{:4f}.{}.epoch{}.pt".format(best_acc, datetime.datetime.now().strftime("%Y%m%d%H%M%S"), best_epoch))
    # plot loss #
    plt.figure()
    plt.plot(train_loss_history,label="train_loss")
    plt.plot(val_loss_history,label="val_loss")
    plt.ylabel("loss")
    plt.xlabel("epochs")
    plt.legend()
    plt.title("loss")
    plt.savefig("loss.png")
    # plot auPR #
    plt.figure()
    plt.plot(train_auPR_history,label="train_auPR")
    plt.plot(val_auPR_history,label="val_auPR")
    plt.ylabel("auPR")
    plt.xlabel("epochs")
    plt.legend()
    plt.title("auPR")
    plt.savefig("auPR.png")
    return model

## Example of how to load your best model ##

In [ ]:
### load our model with best accuracy ###
Best_model_dir = "/home/zhinanlin/Work/AITutor/Final/imageRNN/checkpoints/epoch180.pt"
net.load_state_dict(torch.load(Best_model_dir)) #
net.to(device)
net.eval()

## Example of Testing Loop ##

In [ ]:
y_predictionsw = []
y_probabilities = []
labels_test = []
for i, (inputs, labels) in enumerate(test_dataloader): #
        inputs = inputs.to(device)
        labels_test += list(labels) #
        preds = net(inputs)
        probs = nn.functional.softmax(preds,dim=1)
        _,predictions = torch.max(probs,1)
        y_probabilities.extend(probs.detach().cpu().numpy()[:,1])
        y_predictionsw.extend(predictions.cpu().numpy())